In [8]:
from groq import Groq
from dotenv import load_dotenv
import os
import pprint
import json
import re

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [9]:
base_data_path = r"C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data"


if os.path.exists(base_data_path):
    print("It exists")
else:
    print("Does not exist")

It exists


In [10]:
folders = [
    os.path.join(base_data_path, f) # Gives me the path so I can run datetime on it and get the most recent folder 
    for f in os.listdir(base_data_path) # Grabs all the folders in it
    if os.path.isdir(os.path.join(base_data_path, f)) # Validates that the path actually exists
]


In [11]:
latest_folder = max(folders, key=os.path.getctime)
print("Latest folder:", latest_folder)


Latest folder: C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data\research_on_iphones


In [12]:
report_system_prompt = """
You are a research report generation agent.

You are given:
1. Synthesized research outputs.
2. Gap analysis results.

Your job:

- Produce a structured research report.
- Organize findings by task.
- Integrate identified gaps.
- Provide clear recommendations.
- Do NOT invent new information.
- Stay grounded strictly in provided data.

STRICT RULES:
- Output MUST be valid JSON.
- Use double quotes for all property names.
- No trailing commas.
- No explanation outside <answer> block.

Return ONLY this structure inside <answer> tags:

<answer>
{
  "executive_summary": "",
  "research_sections": [
    {
      "task": "",
      "summary": "",
      "key_findings": []
    }
  ],
  "overall_assessment": {
    "strengths": [],
    "limitations": []
  },
  "identified_gaps": [],
  "recommended_next_steps": []
}
</answer>
"""

In [13]:
report_agent = Groq()


synthesis_path = os.path.join(latest_folder, "synthesis_results.json")
gap_path = os.path.join(latest_folder, "gap_results.json")

In [14]:
with open(synthesis_path, "r") as f:
    synthesized_data = json.load(f)

with open(gap_path, "r") as f:
    gap_data = json.load(f)

In [15]:
combined_input = {
    "synthesized_results": synthesized_data,
    "gap_analysis": gap_data
}

In [16]:
completion = report_agent.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": report_system_prompt},
        {"role": "user", "content": json.dumps(combined_input, indent=2)}
    ],
    stream=False
)

reply = completion.choices[0].message.content

In [17]:

match = re.search(r"<answer>(.*?)</answer>", reply, re.DOTALL)

if not match:
    raise ValueError("No structured report returned")

json_text = match.group(1).strip()

report_results = json.loads(json_text)

with open(os.path.join(latest_folder, "final_report.json"), "w") as f:
    json.dump(report_results, f, indent=4)

print(report_results)

{'executive_summary': 'This research project investigated the environmental and social implications of iPhone production and usage, highlighting challenges such as greenhouse gas emissions, e-waste, and supply chain sustainability concerns. However, labor practices and human rights issues are underrepresented in current research. Identified gaps suggest a need for further investigation into labor practices, individual/community impacts, and political/economic impacts of iPhone production and usage.', 'research_sections': [{'task': 'Research the environmental and social implications of iPhone production and usage (e.g., e-waste, supply chains, labor practices)', 'summary': 'The production and usage of iPhones have significant environmental and social implications, including greenhouse gas emissions, e-waste challenges, and sustainability concerns in the supply chain. However, Apple is making efforts to reduce its carbon footprint through various initiatives.', 'key_findings': [{'finding